In [1]:
import numpy as np
import pandas as pd
import random as rd
from random import randint
import matplotlib.pyplot as plt

# Define Item Number (1, 2, .., 10)
item_number = np.arange(1,11)
print(item_number)

# Define Weight
weight = [20,25,10,5,4,12,17,17,18,19] #np.random.randint(1, 15, size = 10)
print(weight)

# Define Value
value = [100,200,150,120,450,780,1000,65,80,900] #np.random.randint(10, 750, size = 10)
print(weight)

# Define Threshold
knapsack_threshold = 100    #Maximum weight that the bag of thief can hold 

print('The list is as follows:')
print('Item No.   Weight   Value')
for i in range(item_number.shape[0]):
    print('{0}          {1}         {2}\n'.format(item_number[i], weight[i], value[i]))

[ 1  2  3  4  5  6  7  8  9 10]
[20, 25, 10, 5, 4, 12, 17, 17, 18, 19]
[20, 25, 10, 5, 4, 12, 17, 17, 18, 19]
The list is as follows:
Item No.   Weight   Value
1          20         100

2          25         200

3          10         150

4          5         120

5          4         450

6          12         780

7          17         1000

8          17         65

9          18         80

10          19         900



In [ ]:
# Define the number of individuals (solutions) per population
solutions_per_pop = 8

# Define the shape of a population
pop_size = (solutions_per_pop, item_number.shape[0])
print('Population size = {}'.format(pop_size))

# Population initialization
initial_population = np.random.randint(2, size = pop_size)
initial_population = initial_population.astype(int)

# Define the number of generations
num_generations = 50
print('Initial population: \n{}'.format(initial_population))

Population size = (8, 10)
Initial population: 
[[0 1 1 0 0 0 1 0 0 1]
 [0 0 0 1 0 0 1 0 0 1]
 [1 1 0 0 0 0 1 0 0 0]
 [1 1 0 1 0 1 1 1 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 1 1 0 0 1]
 [1 1 1 0 1 1 0 1 1 1]
 [0 0 0 0 1 1 0 0 1 1]]


In [ ]:
def cal_fitness(weight, value, population, threshold):
    fitness = np.empty(population.shape[0])
    for i in range(population.shape[0]):
        S1 = np.sum(population[i] * value)
        S2 = np.sum(population[i] * weight)
        if S2 <= threshold:
            fitness[i] = S1
        else :
            fitness[i] = 0
    return fitness.astype(int)

In [ ]:
def weighted_random_choice(choices):
    max = sum(choices)
    pick = rd.uniform(0, max)
    current = 0
    for i,value in enumerate(choices):
        current =current+value
        if current > pick:
            return i

In [ ]:
def selection(fitness, num_parents, population):
    fitness = list(fitness)
    parents = np.empty((num_parents, population.shape[1]))
    for i in range(num_parents):
        index=weighted_random_choice(fitness)
        parents[i,:] = population[index, :]
    return parents

In [ ]:
def crossover(parents, num_offsprings):
    offsprings = np.empty((num_offsprings, parents.shape[1]))
    crossover_point = int(parents.shape[1]/2)
    crossover_rate = 0.8
    i=0
    while (parents.shape[0] < num_offsprings):
        parent1_index = i%parents.shape[0]
        parent2_index = (i+1)%parents.shape[0]
        x = rd.random()
        if x > crossover_rate:
            continue
        parent1_index = i%parents.shape[0]
        parent2_index = (i+1)%parents.shape[0]
        offsprings[i,0:crossover_point] = parents[parent1_index,0:crossover_point]
        offsprings[i,crossover_point:] = parents[parent2_index,crossover_point:]
        i=+1
    return offsprings

In [ ]:
def mutation(offsprings):
    mutants = np.empty((offsprings.shape))
    mutation_rate = 0.4
    for i in range(mutants.shape[0]):
        random_value = rd.random()
        mutants[i,:] = offsprings[i,:]
        if random_value > mutation_rate:
            continue
        int_random_value = randint(0,offsprings.shape[1]-1)
        if mutants[i,int_random_value] == 0 :
            mutants[i,int_random_value] = 1
        else :
            mutants[i,int_random_value] = 0
    return mutants

In [ ]:
def elitism(mutants, population, fitness, weight, value, threshold):
    elitis = np.empty((mutants.shape))
    elitis = mutants
    parents_max_loc = np.where(fitness == np.max(fitness))[0]
    parents_max_loc_real = parents_max_loc[0]
    fitness_mutants = cal_fitness(weight, value, mutants, threshold)
    mutants_min_loc = np.where(fitness_mutants == np.min(fitness_mutants))
    mutants_min_loc_real = mutants_min_loc[0]
    elitis[mutants_min_loc_real] = population[parents_max_loc_real]
    return elitis

In [ ]:
def optimize(weight, value, population, pop_size, num_generations, threshold):
    parameters, fitness_history = [], []
    num_parents = int(pop_size[0])
    num_offsprings = pop_size[0]
    for i in range(num_generations-1):
        fitness = cal_fitness(weight, value, population, threshold)
        fitness_history.append(fitness)
        print("generation {}".format(i))
        for j in range(pop_size[0]):
            print(population[j,:],fitness[j])
        parents = selection(fitness, num_parents, population)
        offsprings = crossover(parents, num_offsprings)
        mutants = mutation(offsprings)
        #population[0:parents.shape[0], :] = parents
        population = elitism(mutants, population, fitness, weight, value, threshold)

    print('Last generation: \n{}\n'.format(population))
    fitness_last_gen = cal_fitness(weight, value, population, threshold)
    print('Fitness of the last generation: \n{}\n'.format(fitness_last_gen))
    max_fitness = np.where(fitness_last_gen == np.max(fitness_last_gen))
    parameters.append(population[max_fitness[0][0],:])
    return parameters, fitness_history

In [ ]:
parameters, fitness_history = optimize(weight, value, initial_population, pop_size, num_generations, knapsack_threshold)
print('The optimized parameters for the given inputs are: \n{}'.format(parameters))
selected_items = item_number * parameters
print('\nSelected items that will maximize the knapsack without breaking it:')
for i in range(selected_items.shape[1]):
  if selected_items[0][i] != 0:
    {
     print('{}\n'.format(selected_items[0][i]))
     }

generation 0
[0 1 1 0 0 0 1 0 0 1] 2250
[0 0 0 1 0 0 1 0 0 1] 2020
[1 1 0 0 0 0 1 0 0 0] 1300
[1 1 0 1 0 1 1 1 0 0] 2265
[0 0 0 1 0 0 0 0 0 0] 120
[0 0 0 1 0 1 1 0 0 1] 2800
[1 1 1 0 1 1 0 1 1 1] 0
[0 0 0 0 1 1 0 0 1 1] 2210
generation 1
[1. 1. 0. 0. 1. 1. 0. 0. 0. 1.] 2430
[1. 1. 0. 0. 1. 1. 1. 0. 0. 0.] 2530
[0. 0. 0. 1. 0. 1. 1. 0. 0. 1.] 2800
[1. 1. 0. 0. 1. 1. 1. 0. 0. 1.] 3430
[1. 0. 0. 0. 1. 1. 1. 1. 0. 1.] 3295
[0. 0. 0. 1. 0. 1. 1. 0. 0. 1.] 2800
[1. 0. 0. 1. 1. 0. 1. 0. 0. 1.] 2570
[1. 0. 0. 0. 1. 1. 0. 0. 0. 1.] 2230
generation 2
[0. 1. 1. 0. 0. 0. 1. 0. 0. 1.] 2250
[0. 1. 1. 0. 0. 0. 1. 1. 0. 1.] 2315
[0. 0. 0. 0. 1. 1. 0. 0. 1. 1.] 2210
[0. 1. 1. 0. 0. 0. 1. 0. 0. 1.] 2250
[0. 0. 0. 1. 0. 1. 1. 0. 0. 1.] 2800
[1. 1. 0. 0. 1. 1. 1. 0. 0. 1.] 3430
[1. 0. 0. 0. 1. 1. 0. 0. 1. 1.] 2310
[0. 0. 0. 1. 0. 1. 1. 0. 0. 1.] 2800
generation 3
[1. 0. 0. 1. 0. 0. 1. 0. 0. 1.] 2120
[0. 0. 0. 1. 0. 1. 1. 0. 0. 1.] 2800
[1. 0. 0. 0. 1. 1. 1. 1. 0. 1.] 3295
[1. 0. 0. 0. 1. 1. 0. 0. 0. 1.] 2

In [ ]:
print(selected_items[0])

[ 0.  2.  3.  4.  5.  6.  7.  0.  0. 10.]


In [ ]:
nama_makanan = "Nasi, Kentang, Jagung, Ubi, Tempe, Tahu, Telur, Ayam, Sapi, Pisang"
value_makanan = "130, 175, 85, 85, 193, 76, 155, 239, 250, 89"
numpy_nama_makanan = np.array([i for i in nama_makanan.split(',')])
numpy_value_makanan = np.array([int(i) for i in value_makanan.split(',')])
hasil = []
kalori = 0
for i in selected_items[0] :
  if (i!=0) :
    hasil.append(numpy_nama_makanan[int(i)-1])
    kalori = kalori + numpy_value_makanan[int(i)-1]
hasil_string = ""
for i in hasil :
  hasil_string = hasil_string + i + ","


total = []
total.append(hasil_string)
total.append(kalori)

print(total)

[' Kentang, Jagung, Ubi, Tempe, Tahu, Telur, Pisang,', 858]
